Library


In [69]:
import numpy as np
import math

Konstan


In [70]:
EULER = math.exp(1)
THRESHOLD = 0.5

# Fuzzy Class and Functions


## Membership Functions


In [71]:
def sigmf(x, a, c):
    return 1 / (1 + pow(EULER, -c * (x - a)))

def gaussmf(x, mean, sigma):
    return pow(EULER, -((x - mean) ** 2) / (2 * sigma ** 2))

## Fuzzy Sets


In [72]:
def fuzzify_rms(val):
    return {
        'low': sigmf(val, 0.04, -50),
        'medium': gaussmf(val, 0.06, 0.02),
        'high': sigmf(val, 0.1, 50)
    }

def fuzzify_zcr(val):
    return {
        'infrequent': sigmf(val, 5000, -0.05),
        'moderate': gaussmf(val, 8000, 2000),
        'frequent': sigmf(val, 11000, 0.05)
    }

## Mamdani Inference Engine


In [73]:
def fuzzy_mamdani_infer(rms_val, zcr_val):
    rms_set = fuzzify_rms(rms_val)
    zcr_set = fuzzify_zcr(zcr_val)

    rules = [
        ('not_loud', min(rms_set['low'], zcr_set['infrequent'])),
        ('not_loud', min(rms_set['low'], zcr_set['moderate'])),
        ('not_loud', min(rms_set['low'], zcr_set['frequent'])),
        ('not_loud', min(rms_set['medium'], zcr_set['infrequent'])),
        ('not_loud', min(rms_set['medium'], zcr_set['moderate'])),
        ('loud', min(rms_set['medium'], zcr_set['frequent'])),
        ('loud', min(rms_set['high'], zcr_set['infrequent'])),
        ('loud', min(rms_set['high'], zcr_set['moderate'])),
        ('loud', min(rms_set['high'], zcr_set['frequent'])),
    ]

    # Aggregation
    loud_dom = [strength for label, strength in rules if label == 'loud']
    not_loud_dom = [strength for label, strength in rules if label == 'not_loud']

    loud = max(loud_dom) if loud_dom else 0
    not_loud = max(not_loud_dom) if not_loud_dom else 0

    # Defuzzification (centroid approximation)
    output_range = [i / 100.0 for i in range(0, 101)]
    numerator = 0
    denominator = 0
    for o in output_range:
        mu = max(
            min(sigmf(o, 0.6, 20), loud),
            min(sigmf(o, 0.4, -20), not_loud)
        )
        numerator += o * mu
        denominator += mu

    return numerator / denominator if denominator != 0 else 0

## Sugeno Inference Engine


In [74]:
def fuzzy_sugeno_infer(rms_val, zcr_val):
    rms_set = fuzzify_rms(rms_val)
    zcr_set = fuzzify_zcr(zcr_val)

    rules = [
        (min(rms_set['low'], zcr_set['infrequent']), 0.0),
        (min(rms_set['low'], zcr_set['moderate']), 0.0),
        (min(rms_set['low'], zcr_set['frequent']), 0.0),
        (min(rms_set['medium'], zcr_set['infrequent']), 0.0),
        (min(rms_set['medium'], zcr_set['moderate']), 0.0),
        (min(rms_set['medium'], zcr_set['frequent']), 1.0),
        (min(rms_set['high'], zcr_set['infrequent']), 1.0),
        (min(rms_set['high'], zcr_set['moderate']), 1.0),
        (min(rms_set['high'], zcr_set['frequent']), 1.0),
    ]

    numerator = sum([w * z for w, z in rules])
    denominator = sum([w for w, _ in rules])

    return numerator / denominator if denominator != 0 else 0

# Final Classification


In [75]:
def classify_sound(rms_val, zcr_val, method='mamdani'):
    if method == 'mamdani':
        result = fuzzy_mamdani_infer(rms_val, zcr_val)
    elif method == 'sugeno':
        result = fuzzy_sugeno_infer(rms_val, zcr_val)
    else:
        raise ValueError("Method must be either 'mamdani' or 'sugeno'")

    
    if result >= THRESHOLD:
        return 1  
    else:
        return 2  

# Example


Label 1 (Suara Keras)


In [76]:
rms_val = 0.177941
zcr_val = 4608.87156

print("Class (Mamdani):", classify_sound(rms_val, zcr_val, method='mamdani'))
print("Class (Sugeno):", classify_sound(rms_val, zcr_val, method='sugeno'))

Class (Mamdani): 1
Class (Sugeno): 1


Label 2 (Suara Background)


In [77]:
rms_val = 0.018929762
zcr_val = 4480.010417

print("Class (Mamdani):", classify_sound(rms_val, zcr_val, method='mamdani'))
print("Class (Sugeno):", classify_sound(rms_val, zcr_val, method='sugeno'))

Class (Mamdani): 2
Class (Sugeno): 2
